In [1]:
# This is required to run multiple processes on Unity for some reason.
from multiprocessing import set_start_method
set_start_method('spawn', force=True)


import os
os.environ['XLA_FLAGS'] = '--xla_gpu_enable_command_buffer='
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.7'
# os.environ['JAX_PLATFORMS'] = 'cpu'

In [2]:
%matplotlib widget
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import jax
from pathlib import Path
from importlib import reload

src = str(Path('../src').resolve())
if src not in sys.path:
    sys.path.append(src)
import config, data, models, train, evaluate

In [3]:
run_dir = Path("../runs/flexible_hybrid_clip/base_config_20240827_220358")
cfg, model, trainer_state, opt_state, _ = train.load_last_state(run_dir)

cfg['quiet'] = False
cfg['data_subset'] = 'test'
dataset = data.TAPDataset(cfg)


Loading model state from ../runs/flexible_hybrid_clip/base_config_20240827_220358/epoch050


2024-08-28 13:36:44.822402: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version (12.6.20). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


Model contains 178,755 parameters, using 698.26KB memory.


/work/pi_kandread_umass_edu/tss-ml/src/data.py:256: UserWarning: Dropping numerical attributes with 0 variance or NaN values: ['glc_pc_s17', 'wet_pc_s07', 'glc_pc_s08', 'glc_pc_s05', 'glc_pc_s03', 'pnv_pc_s03', 'wet_pc_s06', 'wet_pc_s05', 'glc_pc_s19', 'glc_pc_s07']
  warnings.warn(f"Dropping numerical attributes with 0 variance or NaN values: {cols_to_drop}", UserWarning)
/work/pi_kandread_umass_edu/tss-ml/src/data.py:257: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_df.drop(columns=cols_to_drop, inplace=True)


Data Hash: a81acb9a40087e870fb9bba647932600844d18a66d7df956c53f3c9f7c95140d
Using cached basin dataset file.


Updating Indices:   0%|          | 0/148 [00:00<?, ?it/s]

In [ ]:
cfg['batch_size'] = 1
dataloader = data.TAPDataLoader(cfg, dataset)

for basin, date, batch in dataloader:
    break

(1, 365, 3)

In [11]:
import jax

jax.tree.map(jnp.zeros_like, batch)

{'dynamic': {'era5': Array([[[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          ...,
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]]], dtype=float32),
  'landsat': Array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)},
 'static': Array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [12]:
import jax
import jax.numpy as jnp
import equinox as eqx

def integrated_gradients_flexiblehybrid(model, x, x_baseline, target_neuron_fn, steps=100):
    # Ensure model is in eval mode
    model = eqx.nn.inference_mode(model)

    # Create a path from baseline to input
    def interpolate(alpha):
        return {
            'dynamic': {
                k: x_baseline['dynamic'][k] + alpha * (x['dynamic'][k] - x_baseline['dynamic'][k])
                for k in x['dynamic']
            },
            'static': x_baseline['static'] + alpha * (x['static'] - x_baseline['static'])
            if 'static' in x else None
        }

    alphas = jnp.linspace(0, 1, steps)
    path = [interpolate(alpha) for alpha in alphas]

    # Define the gradient function
    def grad_fn(x):
        key = jax.random.PRNGKey(0)  # Use a fixed key for deterministic behavior
        y = model(x, key)
        return target_neuron_fn(y)

    # Calculate gradients along the path
    grads = jax.vmap(jax.grad(grad_fn))(path)

    # Average the gradients
    avg_grads = jax.tree_map(lambda x: jnp.mean(x, axis=0), grads)

    # Calculate integrated gradients
    integrated_grads = jax.tree_map(lambda g, x_b, x: g * (x - x_b), avg_grads, x_baseline, x)

    return integrated_grads

# def calculate_tsoi(ig, threshold=2e-3):
#     # Sum integrated gradients across features and variables for each time step
#     ig_sum = sum(jnp.sum(jnp.abs(ig['dynamic'][k]), axis=(1, 2)) for k in ig['dynamic'])
    
#     # Calculate differences between consecutive time steps
#     differences = jnp.abs(jnp.diff(ig_sum))
    
#     # Find the first time step where the difference exceeds the threshold
#     mask = differences > threshold
#     if jnp.any(mask):
#         first_significant_step = jnp.argmax(mask)
#     else:
#         first_significant_step = len(ig_sum) - 1
    
#     # Calculate TSOI
#     tsoi = len(ig_sum) - first_significant_step
    
#     return tsoi

def analyze_test_period(model, test_data, target_neuron_fn, threshold=2e-3):
    tsoi_results = []
    
    for sample in test_data:
        x = sample
        x_baseline = jax.tree_map(jnp.zeros_like, x)
        
        ig = integrated_gradients_flexiblehybrid(model, x, x_baseline, target_neuron_fn)
        tsoi = calculate_tsoi(ig, threshold)
        tsoi_results.append(tsoi)
    
    return tsoi_results


x = batch
x_baseline = jax.tree.map(jnp.zeros_like, batch)

target_neuron_fn = lambda y: y[0]  # Assuming you want to focus on the first output

ig = integrated_gradients_flexiblehybrid(model, x, x_baseline, target_neuron_fn)


# # Print results
# print("TSOI for each sample:", tsoi_results)
# print("Average TSOI:", jnp.mean(tsoi_results))
# print("Max TSOI:", jnp.max(tsoi_results))
# print("Min TSOI:", jnp.min(tsoi_results))


/tmp/ipykernel_2897733/3075629776.py:74: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  x_baseline = jax.tree_map(jnp.zeros_like, batch)


TypeError: list indices must be integers or slices, not str